<a name='introduction'></a>
# Проект "Рынок жилья в Калифорнии в 1990 году"

**Задача проекта**: В проекте нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных необходимо предсказать медианную стоимость дома в жилом массиве. Требутеся обучить модель и сделать предсказания на тестовой выборке. Для оценки качества модели используются метрики RMSE, MAE и R2.

*Данные предоставлены Яндекс Практикумом.*

**План проекта:**
1. [Импортирование модулей и данных](#importing)
2. [Предобработка данных](#preprocessing)
3. [Обучение моделей](#training)
    - [Модель с учетом всех признаков](#all_cols_model)
    - [Модель с учетом только количественных признаков](#numeric_cols_model)
4. [Анализ результатов](#analysis)

<a name='importing'></a>
## 1. Импортирование модулей и данных

In [1]:
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StandardScaler, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName('Real Estate Project') \
    .getOrCreate()

In [3]:
df_housing = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema=True)

In [4]:
df_housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

<a name='preprocessing'></a>
## 2. Предобработка данных
[Вернуться во Введение](#introduction)

In [5]:
# Посмотрим на типы данных каждой из колонок рассматриваемого датасета.
print(pd.DataFrame(df_housing.dtypes, columns=['column', 'type']).head(10))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


In [6]:
# Взгляним на различные описательные статистики датасета с помощью describe()
# Для удобства будет считать смотреть по 5 и 4 признаков за раз соответственно. Иначе формат отображения почти нечитаем

df_housing.select('longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms').describe().show()

df_housing.select('population', 'households', 'median_income', 'median_house_value').describe().show()

+-------+-------------------+-----------------+------------------+------------------+------------------+
|summary|          longitude|         latitude|housing_median_age|       total_rooms|    total_bedrooms|
+-------+-------------------+-----------------+------------------+------------------+------------------+
|  count|              20640|            20640|             20640|             20640|             20433|
|   mean|-119.56970445736148| 35.6318614341087|28.639486434108527|2635.7630813953488| 537.8705525375618|
| stddev|  2.003531723502584|2.135952397457101| 12.58555761211163|2181.6152515827944|421.38507007403115|
|    min|            -124.35|            32.54|               1.0|               2.0|               1.0|
|    max|            -114.31|            41.95|              52.0|           39320.0|            6445.0|
+-------+-------------------+-----------------+------------------+------------------+------------------+

+-------+------------------+-----------------+--------

In [7]:
# Посчитаем среднее количество квартир в доме для заполнения пропущенных значений
mean_bedrooms = df_housing.select(F.mean('total_bedrooms')).collect()[0][0]
print("Среднее количество комнат: {:.2f}".format(mean_bedrooms))

df_housing = df_housing.na.fill(mean_bedrooms)

Среднее количество комнат: 537.87


In [8]:
# Посмотрим на количество пропущенных значений по каждой из колонок
columns = df_housing.columns

df_housing.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|             0|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



In [9]:
target = 'median_house_value'

def create_pipeline(all_cols=True):
    # Создадим модель линейной регрессии с помощью пайплана
    # all_cols отвечает за (не)включение всех признаков
    stages = []
    
    # При необходимости - добавление категориальных признаков в этапе обработки пайплайнов
    if all_cols:
        indexer = StringIndexer(inputCol='ocean_proximity', outputCol='ocean_proximity_idx')
        encoder = OneHotEncoder(inputCol='ocean_proximity_idx', outputCol='categorical_features')
        stages += [indexer, encoder]

    # Осуществим масштабирование количественных признаков
    numeric_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', \
                    'total_bedrooms', 'population', 'households', 'median_income']

    numeric_assembler = VectorAssembler(inputCols=numeric_cols, outputCol='numeric_features')
    stages += [numeric_assembler]

    scaler = StandardScaler(inputCol='numeric_features', outputCol='numeric_features_scaled')
    stages += [scaler]

    # Соберем вместе категориальные и количественные признаки
    all_features = ['numeric_features_scaled']
    
    # Добавление в обработку категориальных признаков, если требуется
    if all_cols: 
        all_features += ['categorical_features']

    final_assembler = VectorAssembler(inputCols=all_features, outputCol='features')
    stages += [final_assembler]

    lr = LinearRegression(maxIter=100, loss='squaredError', featuresCol='features', labelCol=target)
    stages += [lr]
    
    return Pipeline(stages=stages)

In [10]:
# Разделим датасет на тренировочную и тестовую выборки
RANDOM_SEED = 42

train_data, test_data = df_housing.randomSplit([.75, .25], seed=RANDOM_SEED)
print(train_data.count(), test_data.count())

15500 5140


<a name='training'></a>
## 3. Обучение моделей
[Вернуться во Введение](#introduction)

<a name='all_cols_model'></a>
### 3. 1. Модель с учетом всех признаков

In [11]:
pipeline = create_pipeline()
lr_model = pipeline.fit(train_data)

23/03/23 09:29:56 WARN Instrumentation: [cfd51b8f] regParam is zero, which might cause numerical instability and overfitting.
23/03/23 09:29:56 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/03/23 09:29:56 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/03/23 09:29:57 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/03/23 09:29:57 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [12]:
pred = lr_model.transform(test_data)
predictedLabels = pred.select('prediction', target)
predictedLabels.show(10)

+------------------+------------------+
|        prediction|median_house_value|
+------------------+------------------+
| 149740.0781819867|          103600.0|
| 217279.7178173284|          106700.0|
|124444.53000378748|           73200.0|
|125874.49652704736|           78300.0|
|194153.01353182225|           90100.0|
| 151690.8616671795|           67000.0|
|185550.46387161547|           86400.0|
|163562.46115427604|           70500.0|
| 142686.6072390182|           60000.0|
|158727.83188629802|           75500.0|
+------------------+------------------+
only showing top 10 rows



In [13]:
train_summary = lr_model.stages[-1].summary
print("RMSE: {:.2f}".format(train_summary.rootMeanSquaredError))
print("r2: {:.2f}".format(train_summary.r2))
print("MAE: {:.2f}".format(train_summary.meanAbsoluteError))

RMSE: 68486.08
r2: 0.65
MAE: 49640.62


Создадим далее модель линейной регрессии, которая будет учитывать только количественные признаки.

<a name='numeric_cols_model'></a>
### 3. 2. Модель с учетом только количественных признаков

In [14]:
pipeline = create_pipeline(all_cols=False)
lr_model = pipeline.fit(train_data)

23/03/23 09:30:01 WARN Instrumentation: [5ad81511] regParam is zero, which might cause numerical instability and overfitting.


In [15]:
pred = lr_model.transform(test_data)
predictedLabels = pred.select('prediction', target)
predictedLabels.show(10)

+------------------+------------------+
|        prediction|median_house_value|
+------------------+------------------+
| 99974.57187516708|          103600.0|
|190948.50971714407|          106700.0|
|  73994.5681234384|           73200.0|
| 77461.39460649993|           78300.0|
|161772.96565529192|           90100.0|
|119226.41383205727|           67000.0|
| 155771.3090367224|           86400.0|
|131221.58270589542|           70500.0|
|111532.55901558604|           60000.0|
|127280.60082717985|           75500.0|
+------------------+------------------+
only showing top 10 rows



In [16]:
train_summary = lr_model.stages[-1].summary
print("RMSE: {:.2f}".format(train_summary.rootMeanSquaredError))
print("r2: {:.2f}".format(train_summary.r2))
print("MAE: {:.2f}".format(train_summary.meanAbsoluteError))

RMSE: 69432.15
r2: 0.64
MAE: 50771.85


In [17]:
# Окончание Spark-сессии
spark.stop()

<a name='analysis'></a>
# 4. Анализ результатов и выводы
[Вернуться во Введение](#introduction)

<br>**Результаты**:

Разница в результатах в зависимости от использования модели, учитывающей все признаки - категориальные и количественные - и учитывающей только количественные, не существенна. Показатели средней квадратичной ошибки различаются на тысячу долларов - около 68 000 и 69 000 долл. соответственно. Вероятно, это связано с использованием определенного случайного сида при разделении датасета на тренировочную и тестовую выборки. Тем не менее, при использовании модели, учитывающей все признаки, значение RMSE и MAE ниже, чем в случае с моделью, учитывающей лишь количественные признаки.

<br>**Вывод**:

Ввиду этого рекомендуется использование модели, которая осуществляет учет как количественных, так и категориальных параметров.